<font color='tomato'><font color="#CC3D3D"><p>
# Neural Collaborative Filtering (NCF): Basic Model

- 행렬분해(MF)의 MLP 변환    

<img src='NCF_Basics.jpeg' align="left" width=800>

- NCF 아키텍처   

<img src='NCF_Basic_architecture.jpeg' align="left" width=800/>

### Import modules

In [1]:
import pandas as pd
import numpy as np
import pickle, random, os
import warnings; warnings.filterwarnings("ignore")
import matplotlib.pylab as plt
%matplotlib inline

from msr.python_splitters import (
    python_random_split, 
    python_chrono_split, 
    python_stratified_split
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.13.0


### Set random seeds to make your results reproducible

In [2]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")

    np.random.seed(1)
    random.seed(2)
    tf.compat.v1.set_random_seed(3)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET") 
   
reset_seeds()

RANDOM SEEDS RESET


### Step 1: Load and process the data

##### Read data

In [3]:
movies = pd.read_csv('movies.csv')    # 영화 정보
ratings = pd.read_csv('ratings.csv')  # 사용자가 영화에 대해 남긴 평점 데이터

movies.shape, ratings.shape

((9724, 3), (100836, 4))

##### Split data into train & test set 

In [4]:
#train, test = python_random_split(ratings, ratio=0.75)
train, test = train_test_split(ratings, test_size=0.25, shuffle=True, random_state=0)

### Step 2: Define the model architecture

##### Stack layers from input to output

In [5]:
# Variable 초기화 
K = 200                             # Latent factor 수 
mu = train.rating.mean()            # 전체 평균 
M = ratings.userId.nunique() + 1    # Number of users
N = ratings.movieId.nunique() + 1   # Number of movies

M, N

(611, 9725)

In [6]:
ratings.userId

0           1
1           1
2           1
3           1
4           1
         ... 
100831    610
100832    610
100833    610
100834    610
100835    610
Name: userId, Length: 100836, dtype: int64

In [7]:
user = keras.Input(shape=(1, ))                          # User input: userID
item = keras.Input(shape=(1, ))                          # Item input: movieId

P_embedding = keras.layers.Embedding(M, K)(user)         # M: Number of users, K: Latent factor 수 
Q_embedding = keras.layers.Embedding(N, K)(item)         # M: Number of movies, K: Latent factor 수 

user_bias = keras.layers.Embedding(M, 1)(user)           # User bias term 
item_bias = keras.layers.Embedding(N, 1)(item)           # Item bias term

R = keras.layers.Dot(axes=2)([P_embedding, Q_embedding]) # 행렬곱: P * Q 
R = keras.layers.Add()([R, user_bias, item_bias])
R = keras.layers.Flatten()(R)

#R = Dense(32, activation='relu')(R)
#R = Dense(16, activation='relu')(R)
#R = Dense(1, activation='linear')(R)

model = keras.Model(inputs=[user, item], outputs=R)

##### Summarize & visualize the model

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 200)               122200    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 200)               1945000   ['input_2[0][0]']             
                                                                                              

In [9]:
# 연결 그래프 시각화: graphviz와 pydot 설치 필요 (graphviz 다운로드하고 설치한 후 pip install pydot)
keras.utils.plot_model(model, show_shapes=True, rankdir='BT')

### Step 3: Choose the optimizer and the cost function

In [10]:
# loss, optimizer, metrics 설정
model.compile(
    loss='mse',
    optimizer=keras.optimizers.legacy.Adam(), 
    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')]
)

### Step 4: Train the model

##### Set learning conditions & fit the model 

In [11]:
hist = model.fit(
    x=[train.userId, train.movieId],
    y=train.rating - mu,
    validation_split=0.1,
    batch_size=256, 
    epochs=50, 
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
    shuffle=False,
    verbose=2
)

Epoch 1/50
266/266 - 1s - loss: 1.0352 - rmse: 1.0174 - val_loss: 0.9786 - val_rmse: 0.9892 - 1s/epoch - 5ms/step
Epoch 2/50
266/266 - 1s - loss: 0.8385 - rmse: 0.9157 - val_loss: 0.8381 - val_rmse: 0.9155 - 1s/epoch - 4ms/step
Epoch 3/50
266/266 - 1s - loss: 0.5374 - rmse: 0.7331 - val_loss: 0.7611 - val_rmse: 0.8724 - 1s/epoch - 4ms/step
Epoch 4/50
266/266 - 1s - loss: 0.3014 - rmse: 0.5490 - val_loss: 0.7497 - val_rmse: 0.8658 - 1s/epoch - 4ms/step
Epoch 5/50
266/266 - 1s - loss: 0.1615 - rmse: 0.4019 - val_loss: 0.7566 - val_rmse: 0.8698 - 1s/epoch - 4ms/step
Epoch 6/50
266/266 - 1s - loss: 0.0882 - rmse: 0.2969 - val_loss: 0.7660 - val_rmse: 0.8752 - 1s/epoch - 5ms/step
Epoch 7/50
266/266 - 1s - loss: 0.0497 - rmse: 0.2229 - val_loss: 0.7750 - val_rmse: 0.8803 - 1s/epoch - 5ms/step
Epoch 8/50
266/266 - 1s - loss: 0.0296 - rmse: 0.1721 - val_loss: 0.7810 - val_rmse: 0.8838 - 1s/epoch - 5ms/step
Epoch 9/50
266/266 - 1s - loss: 0.0189 - rmse: 0.1374 - val_loss: 0.7862 - val_rmse: 0.8

##### Visualize training history 

In [12]:
# Plot RMSE

plt.plot(hist.history['rmse'], label="train")
plt.plot(hist.history['val_rmse'], label="valid")
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.legend()
plt.show()

### Step 5: Make predictions

##### Evaluate the model

In [13]:
# 평가데이터에 대한 RMSE 측정

y_pred = model.predict([test.userId, test.movieId]) + mu
y_pred = y_pred.flatten()
y_true = test.rating

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'RMSE = {rmse:.3f}')

788/788 [==============================] - 0s 237us/step
RMSE = 0.901


##### Make recommendations

In [14]:
# Top-N 영화 추천

N = 10
targetUser = 276

# 본 영화 제외
seen_movies = ratings.query('userId == @targetUser').movieId.unique()
movie_ids = pd.Series(np.array(list(set(ratings.movieId.unique()) - set(seen_movies))))

In [15]:
movie_ids.shape

(9683,)

In [16]:
# 평점 예측 대상 영화수만큼 userId 반복
user_ids = pd.Series(np.array([targetUser]*len(movie_ids)))
user_ids.shape

(9683,)

In [17]:
# 평점 예측
predictions = model.predict([user_ids, movie_ids]) + mu

# 가장 평점이 높은 N개의 영화를 추천
pd.DataFrame({'movieId':movie_ids, 'pred': predictions.flatten()}).sort_values(by='pred', ascending=False).head(N)

  1/303 [..............................] - ETA: 2s

303/303 [==============================] - 0s 280us/step


,movieId,pred
758,793,5.222309
2312,2353,5.092922
1244,1285,5.045713
2153,2194,4.960724
775,811,4.957056
1875,1916,4.944647
1444,1485,4.926727
1996,2037,4.924952
486,506,4.915617
900,938,4.908442


###  Step 6: Save the model

In [18]:
# pickle 형식으로 저장
# with open('ncf_model.pkl', 'wb') as f:
#    pickle.dump(model, f)

# 추후 저장한 모형 불러올 때: 
# model = pd.read_pickle('ncf_model.pkl')

<font color="#CC3D3D"><p>
# End